In [159]:
# Imports

import open3d as o3d
import numpy as np
import cv2
import numpy as np 
from ultralytics import YOLO
import copy
import math

In [160]:
# Data Path

model_data = r"C:\Hackathon\3D bone mapping\forearm_Bones.glb"
image_data = r"C:\Hackathon\3D bone mapping\working_code\test\test_6.png"

In [161]:
# Model and Image

mesh = o3d.io.read_triangle_mesh(model_data)
img = cv2.imread(image_data)

scale_percent = 50  # Resize to 50% of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
resized_img = cv2.resize(img, (width, height))

In [162]:
# Per-processing Model

mesh.remove_duplicated_vertices()
mesh.remove_degenerate_triangles()
mesh.remove_duplicated_triangles()
mesh.remove_non_manifold_edges()
mesh.remove_unreferenced_vertices()

mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())
mesh.translate(-mesh.get_center())
mesh.compute_vertex_normals()

R1 = mesh.get_rotation_matrix_from_axis_angle([np.pi, 0, 0])
R2 = mesh.get_rotation_matrix_from_axis_angle([0, -np.pi / 2, 0])

R = R2 @ R1
mesh.rotate(R2, center=(0, 0, 0))

TriangleMesh with 485454 points and 970900 triangles.

In [163]:
# Bounding Box around model 

bbox = mesh.get_axis_aligned_bounding_box()
bbox.color = (1, 0, 0)

min_bound = bbox.get_min_bound()
max_bound = bbox.get_max_bound()
size = max_bound - min_bound

print(f"Width:  {size[0]:.4f}")
print(f"Height: {size[1]:.4f}")
print(f"Depth:  {size[2]:.4f}")
print("Model center:", mesh.get_center())

o3d.visualization.draw_geometries([mesh, bbox])

Width:  0.2318
Height: 1.0000
Depth:  0.1647
Model center: [ 3.4601e-14  1.0308e-12  6.0746e-13]


In [164]:
# Divide Mesh to make Clusters 

plane_height, plane_depth = size[1], size[2]
plane_thickness = 0.001

vertical_plane = o3d.geometry.TriangleMesh.create_box(
    width=plane_thickness,
    height=plane_height,
    depth=plane_depth
)

vertical_plane.translate((-plane_thickness/2, -plane_height/2, -plane_depth/2))

center_x = (min_bound[0] + max_bound[0]) / 2
vertical_plane.translate((center_x, 0, 0))

angle_from_x = 94.11222884471846
tilt_angle = angle_from_x - 90
angle_rad = np.deg2rad(-tilt_angle)

R = vertical_plane.get_rotation_matrix_from_axis_angle([0, 0, angle_rad])
vertical_plane.rotate(R, center=vertical_plane.get_center())

shift_amount = -0.015
vertical_plane.translate((shift_amount, 0, 0))

vertical_plane.paint_uniform_color([1, 0.7, 0.3])

o3d.visualization.draw_geometries([mesh, vertical_plane])


In [165]:
# Create Clusters

plane_normal = R @ np.array([1.0, 0.0, 0.0])
plane_normal /= np.linalg.norm(plane_normal)

plane_center = vertical_plane.get_center()
points = np.asarray(mesh.vertices)

signed_distances = np.dot(points - plane_center, plane_normal)

mask_above = signed_distances > 0
mask_below = signed_distances <= 0

mesh_ulna = mesh.select_by_index(np.where(mask_above)[0].tolist())
mesh_radius = mesh.select_by_index(np.where(mask_below)[0].tolist())

mesh_ulna.paint_uniform_color([0.2, 0.8, 1.0])
mesh_radius.paint_uniform_color([1.0, 0.4, 0.4])

o3d.visualization.draw_geometries([mesh_radius, mesh_ulna])

In [166]:
# Model Landmarks 

model_landmarks = {
    "ulna_head": (0.058, 0.35, 0.0),
    "ulna_tail": (0.0028, -0.43, 0.0),
    "radius_head": (-0.013, 0.35, 0.0),
    "radius_tail": (-0.07, -0.42, 0.0)
}

In [167]:
# Landmarks Marking Funcation

def draw_cylinder_between(p1, p2, radius=0.01, color=[1, 0, 0]):
    p1, p2 = np.array(p1), np.array(p2)
    axis = p2 - p1
    height = np.linalg.norm(axis)
    midpoint = (p1 + p2) / 2

    cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=height)
    cylinder.paint_uniform_color(color)
    cylinder.compute_vertex_normals()

    z_axis = np.array([0, 0, 1])
    axis_norm = axis / np.linalg.norm(axis)
    v = np.cross(z_axis, axis_norm)
    c = np.dot(z_axis, axis_norm)

    if np.linalg.norm(v) < 1e-6:
        R = np.eye(3)
    else:
        vx = np.array([
            [0, -v[2], v[1]],
            [v[2], 0, -v[0]],
            [-v[1], v[0], 0]
        ])
        R = np.eye(3) + vx + vx @ vx * ((1 - c) / (np.linalg.norm(v) ** 2))

    cylinder.rotate(R, center=(0, 0, 0))
    cylinder.translate(midpoint)
    return cylinder

In [168]:
# Landmark Marking

ulna_line = draw_cylinder_between(model_landmarks["ulna_head"], model_landmarks["ulna_tail"], radius=0.01, color=[1, 0, 0])
radius_line = draw_cylinder_between(model_landmarks["radius_head"], model_landmarks["radius_tail"], radius=0.01, color=[0, 1, 0])

In [169]:
# Visualize Model with Landmarks 

o3d.visualization.draw_geometries([mesh, ulna_line, radius_line])

In [170]:
# Display Image 

cv2.imshow("X-ray", img)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [171]:
# Get Pixel Coordintes ( Manually )

clone = img.copy()

labels = ['ulna head', 'ulna tail', 'radius head', 'radius tail']
points = []
index = 0
Xray_landmark = {}

def click_landmarks(event, x, y, flags, param):
    global index, Xray_landmark

    if event == cv2.EVENT_LBUTTONDOWN and index < len(labels):
        label = labels[index]
        points.append((x, y))
        Xray_landmark[label] = (x, y)

        cv2.circle(clone, (x, y), 5, (0, 0, 255), -1)
        cv2.putText(clone, label, (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        index += 1
        cv2.imshow("Image", clone)

        if index == len(labels):
            h, w = clone.shape[:2]
            cx, cy = w // 2, h // 2

            for k in Xray_landmark:
                x, y = Xray_landmark[k]
                Xray_landmark[k] = (x - cx, cy - y)

            print("\n--> Centered Coordinates (origin at image center):")
            for key, val in Xray_landmark.items():
                print(f"{key} → {val}")

cv2.imshow("Image", clone)
cv2.setMouseCallback("Image", click_landmarks)
cv2.waitKey(0)
cv2.destroyAllWindows()


--> Centered Coordinates (origin at image center):
ulna head → (92, 260)
ulna tail → (24, -117)
radius head → (59, 248)
radius tail → (-30, -124)


In [172]:
model = YOLO(r"C:\Users\Rounak\Downloads\best(1).pt")
results = model(img)
boxes = results[0].boxes.xyxy.cpu().numpy()
h, w = img.shape[:2]
cx = w // 2
Xray_breaks = {}

if len(boxes) == 0:
    print("❌ No fractures detected.")
elif len(boxes) == 1:
    x1, y1, x2, y2 = boxes[0]
    x_center = (x1 + x2) / 2
    y_center = (y1 + y2) / 2
    if x_center < cx:
        Xray_breaks['radius break'] = (int(x_center - cx), int((h // 2) - y_center))
        print("→ radius fracture")
    else:
        Xray_breaks['ulna break'] = (int(x_center - cx), int((h // 2) - y_center))
        print("→ ulna fracture")
else:
    for box in boxes:
        x1, y1, x2, y2 = box
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        if x_center < cx and 'radius break' not in Xray_breaks:
            Xray_breaks['radius break'] = (int(x_center - cx), int((h // 2) - y_center))
            print("→ radius fracture")
        elif x_center >= cx and 'ulna break' not in Xray_breaks:
            Xray_breaks['ulna break'] = (int(x_center - cx), int((h // 2) - y_center))
            print("→ ulna fracture")

print("\n--> Detected Fracture Points (centered):")
for k, v in Xray_breaks.items():
    print(f"{k} → {v}")


0: 640x288 1 break, 71.2ms
Speed: 3.5ms preprocess, 71.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 288)
→ radius fracture

--> Detected Fracture Points (centered):
radius break → (-42, -3)


In [173]:
def angle_from_negative_x(p1, p2, center=(0, 0)):

    x1, y1 = p1[0] - center[0], p1[1] - center[1]
    x2, y2 = p2[0] - center[0], p2[1] - center[1]
    dx, dy = x2 - x1, y2 - y1

    angle_rad = math.atan2(dy, dx)
    angle_deg = math.degrees(angle_rad) % 360
    angle_from_neg_x = (angle_deg - 180) % 360

    if angle_from_neg_x <= 90:
        return -(90 - angle_from_neg_x)
    
    elif angle_from_neg_x <= 180:
        return angle_from_neg_x - 90
    
    elif angle_from_neg_x <= 270:
        return 270 - angle_from_neg_x
    
    return 360 - angle_from_neg_x

In [174]:
def get_split_ratio(point_top, point_bottom, split_point):

    y_top, y_bottom, y_split = point_top[1], point_bottom[1], split_point[1]

    if y_top < y_bottom:
        y_top, y_bottom = y_bottom, y_top

    total_height = y_top - y_bottom
    return 1 - (y_top - y_split) / total_height if total_height else 0

In [175]:
def create_angle_mesh(mesh, angles, split_ratio):

    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    min_y = vertices[:, 1].min()
    max_y = vertices[:, 1].max()

    mid_y = min_y + (max_y - min_y) * split_ratio
    top_mask = vertices[:, 1] >= mid_y
    bottom_mask = ~top_mask

    def rotate_part(mask, angle_deg, center_y):

        indices = np.where(mask)[0]
        sub_vertices = np.copy(vertices[indices])

        index_map = -np.ones(len(vertices), dtype=int)
        index_map[indices] = np.arange(len(indices))

        tri_mask = np.all(mask[triangles], axis=1)

        sub_triangles = triangles[tri_mask]
        mapped_triangles = index_map[sub_triangles]

        angle_rad = np.radians(angle_deg)

        R = mesh.get_rotation_matrix_from_axis_angle([0, 0, angle_rad])
        center = [sub_vertices[:, 0].mean(), center_y, sub_vertices[:, 2].mean()]
        rotated = (R @ (sub_vertices - center).T).T + center

        sub_mesh = o3d.geometry.TriangleMesh()
        sub_mesh.vertices = o3d.utility.Vector3dVector(rotated)
        sub_mesh.triangles = o3d.utility.Vector3iVector(mapped_triangles)
        sub_mesh.compute_vertex_normals()
        
        return sub_mesh

    top_mesh = rotate_part(top_mask, angles[0], mid_y)
    bottom_mesh = rotate_part(bottom_mask, -angles[1], mid_y)
    return top_mesh + bottom_mesh

In [ ]:
def drill_fracture_at_split(mesh, img, boxes, split_ratio, size):
    import cv2
    import numpy as np
    from scipy.spatial import cKDTree

    vertices = np.asarray(mesh.vertices)
    mask = np.ones(len(vertices), dtype=bool)
    tree = cKDTree(vertices)

    h_img, w_img = img.shape[:2]
    x_step = size[0] / w_img
    cuboid_depth = 0.07
    radius = max(x_step, cuboid_depth)

    min_y = vertices[:, 1].min()
    max_y = vertices[:, 1].max()
    mid_y = min_y + (max_y - min_y) * split_ratio

    crop = np.zeros((h_img, w_img), dtype=np.uint8)
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(crop, (x1, y1), (x2, y2), 255, -1)

    dilated = cv2.dilate(crop, np.ones((7, 7), np.uint8), iterations=1)
    white_pixels = np.argwhere(dilated == 255)

    x_ratios = (white_pixels[:, 1] - w_img / 2) / (w_img / 2)
    center_xs = x_ratios * size[0] / 2
    center_ys = np.full_like(center_xs, mid_y)
    center_zs = np.zeros_like(center_xs)
    centers = np.stack((center_xs, center_ys, center_zs), axis=1)

    for center in centers:
        idxs = tree.query_ball_point(center, r=radius)
        if not idxs:
            continue
        pts = vertices[idxs]
        
        valid_mask = (
            (np.abs(pts[:, 0] - center[0]) <= x_step / 2) &
            (np.abs(pts[:, 1] - center[1]) <= 0.01) &
            (np.abs(pts[:, 2] - center[2]) <= cuboid_depth / 2)
        )
        mask[np.array(idxs)[valid_mask]] = False

    return mesh.select_by_index(np.where(mask)[0])


In [177]:
if "ulna break" in Xray_breaks and "ulna head" in Xray_landmark and "ulna tail" in Xray_landmark:
    split_ratio_ulna = get_split_ratio(
        Xray_landmark["ulna head"], Xray_landmark["ulna tail"], Xray_breaks["ulna break"]
    )
    
    angle_ulna = angle_from_negative_x(Xray_landmark["ulna head"], Xray_breaks["ulna break"])
    drilled_ulna = drill_fracture_at_split(mesh_ulna, img, boxes, split_ratio_ulna, size)
    fractured_ulna = create_angle_mesh(drilled_ulna, angles=[angle_ulna, angle_ulna], split_ratio=split_ratio_ulna)
else:
    fractured_ulna = mesh_ulna


if "radius break" in Xray_breaks and "radius head" in Xray_landmark and "radius tail" in Xray_landmark:
    split_ratio_radius = get_split_ratio(
        Xray_landmark["radius head"], Xray_landmark["radius tail"], Xray_breaks["radius break"]
    )
    
    angle_radius = angle_from_negative_x(Xray_landmark["radius head"], Xray_breaks["radius break"])
    drilled_radius = drill_fracture_at_split(mesh_radius, img, boxes, split_ratio_radius, size)
    fractured_radius = create_angle_mesh(drilled_radius, angles=[angle_radius, angle_radius], split_ratio=split_ratio_radius)
else:
    fractured_radius = mesh_radius


In [178]:
o3d.visualization.draw_geometries([fractured_ulna, fractured_radius])